In [ ]:
# Copyright: Abdul Hasib Uddin <abdulhasibuddin@gmail.com>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
work_dir = "drive/My Drive/Sentiment_Analysis/"
#impl_type = "GRU"
#dataset_name = "BanglaSentiment_Bin_NafisAlvi"

In [ ]:

import numpy as np
import pandas as pd
import pickle
import numpy as np
import random
import time
import csv
import os
#os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler, LambdaCallback
from IPython.display import display
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import CSVLogger

In [ ]:
#from google.colab import files
'''
files.upload()
files.upload()
#'''

with open(work_dir+'BanglaSentiment_Bin_Data.txt', 'r', encoding="utf8") as f:
    dataset = f.read()
with open(work_dir+'BanglaSentiment_Bin_Label.txt', 'r', encoding="utf8") as f:
    labelset = f.read()

print('Done file uploading!')

Done file uploading!


In [ ]:
#dataset

In [ ]:
list_data = dataset.split("\n")
list_label = labelset.split("\n")

print("len(list_data) =", len(list_data))
print("len(list_label) =", len(list_label))

len(list_data) = 7000
len(list_label) = 7000


In [ ]:
list_data[:5]

[' সাকিব মাশরাফির যোগ্য সহচর  মাশরাফি এই টিমের নিউক্লিয়াস হইলে সাকিব এই টিমের পাওয়ার হাউজ অর্থাৎ মাইট্রোকন্ড্রিয়া ',
 ' কথাটা সত্য বলেছেন এক ব্যক্তির ক্ষমতা তাই আওয়ামীলীগ বলে লাভ কি প্রধানমন্ত্রী কথা বলাই ভালসাধারণ সম্পাদক উনি বানাতে পারে উনি আবার ',
 ' ওরে মারার জন্য যা লাগে তাই দিব ',
 ' তামিম ভাই আপনাকে ধন্যবাদ একশ রান করার জন্য ',
 ' কুরআন শরিফে খুব সুন্দার করে এর বর্ননা দেওয়া হয়েছে ']

In [ ]:
print(list_label[0])
print(list_label[-1])

positive
negative


In [ ]:
list_label[:10]

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

In [ ]:
#'''
from keras.preprocessing.text import Tokenizer

# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(list_data)
# summarize what was learned
#print(t.word_counts)
#print(t.document_count)
#print(t.word_index)
#print(t.word_docs)
# integer encode documents
encoded_dataset = t.texts_to_matrix(list_data, mode='count')
#print(encoded_dataset)
print(encoded_dataset[:5])
print(encoded_dataset[0][10])
#'''

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
0.0


In [ ]:
'''
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence
# define the document
#text = 'The quick brown fox jumped over the lazy dog.'
# estimate the size of the vocabulary
words = set(text_to_word_sequence(dataset))
vocab_size = len(words)
print(vocab_size)
# integer encode the document
#encoded_dataset = hashing_trick(dataset, round(vocab_size*1.3), hash_function='md5')

encoded_dataset = []
for data in list_data:
    encoded_dataset.append(hashing_trick(data, round(vocab_size*1.3), hash_function='md5'))

print("len(encoded_dataset) =", len(encoded_dataset))
print("len(encoded_dataset[0]) =", len(encoded_dataset[0]))
print("len(encoded_dataset[-1]) =", len(encoded_dataset[-1]))
print(encoded_dataset[:5])
#'''

'\nfrom keras.preprocessing.text import hashing_trick\nfrom keras.preprocessing.text import text_to_word_sequence\n# define the document\n#text = \'The quick brown fox jumped over the lazy dog.\'\n# estimate the size of the vocabulary\nwords = set(text_to_word_sequence(dataset))\nvocab_size = len(words)\nprint(vocab_size)\n# integer encode the document\n#encoded_dataset = hashing_trick(dataset, round(vocab_size*1.3), hash_function=\'md5\')\n\nencoded_dataset = []\nfor data in list_data:\n    encoded_dataset.append(hashing_trick(data, round(vocab_size*1.3), hash_function=\'md5\'))\n\nprint("len(encoded_dataset) =", len(encoded_dataset))\nprint("len(encoded_dataset[0]) =", len(encoded_dataset[0]))\nprint("len(encoded_dataset[-1]) =", len(encoded_dataset[-1]))\nprint(encoded_dataset[:5])\n#'

In [ ]:
print("np.array(encoded_dataset).shape =", np.array(encoded_dataset).shape)
print("len(encoded_dataset[0]) =", len(encoded_dataset[0]))
print("len(encoded_dataset[-1]) =", len(encoded_dataset[-1]))

np.array(encoded_dataset).shape = (7000, 21889)
len(encoded_dataset[0]) = 21889
len(encoded_dataset[-1]) = 21889


In [ ]:
'''
zero_pad = np.zeros(encoded_dataset.shape)
zero_pad[:a.shape[0],:a.shape[1]] = a
#'''

'\nzero_pad = np.zeros(encoded_dataset.shape)\nzero_pad[:a.shape[0],:a.shape[1]] = a\n#'

In [ ]:
'''
padded_dataset = np.pad(encoded_dataset, [(0, 1), (0, 1)], mode='constant', constant_values=0)
print("len(padded_dataset[0]) =", len(padded_dataset[0]))
print("len(padded_dataset[-1]) =", len(padded_dataset[-1]))
#'''

'\npadded_dataset = np.pad(encoded_dataset, [(0, 1), (0, 1)], mode=\'constant\', constant_values=0)\nprint("len(padded_dataset[0]) =", len(padded_dataset[0]))\nprint("len(padded_dataset[-1]) =", len(padded_dataset[-1]))\n#'

In [ ]:
dataset_positive = []
labelset_positive = []
dataset_negative = []
labelset_negative = []

for data,label in zip(encoded_dataset,list_label):
    if label == 'positive':
        dataset_positive.append(data)
        labelset_positive.append(1)
    elif label == 'negative':
        dataset_negative.append(data)
        labelset_negative.append(0)

print("len(dataset_positive) =", len(dataset_positive))
print("len(labelset_positive) =", len(labelset_positive))
print("len(dataset_negative) =", len(dataset_negative))
print("len(labelset_negative) =", len(labelset_negative))

len(dataset_positive) = 3500
len(labelset_positive) = 3500
len(dataset_negative) = 3500
len(labelset_negative) = 3500


In [ ]:
#print()

In [ ]:
dataset_stratified = []
labelset_stratified = []

for i in range(len(labelset_positive)):
    dataset_stratified.append(dataset_positive[i])
    dataset_stratified.append(dataset_negative[i])

    labelset_stratified.append(labelset_positive[i])
    labelset_stratified.append(labelset_negative[i])

print("len(dataset_stratified) =", len(dataset_stratified))
print("len(labelset_stratified) =", len(labelset_stratified))

len(dataset_stratified) = 7000
len(labelset_stratified) = 7000


In [ ]:
print(labelset_stratified[:10])

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [ ]:
'''
TRAIN_TEST_SPLIT = int(len(labelset_stratified)*0.5)
print("TRAIN_TEST_SPLIT =", TRAIN_TEST_SPLIT)

train_val_dataset = dataset_stratified[TRAIN_TEST_SPLIT:]
test_dataset = dataset_stratified[:TRAIN_TEST_SPLIT]

train_val_labelset = labelset_stratified[TRAIN_TEST_SPLIT:]
test_labelset = labelset_stratified[:TRAIN_TEST_SPLIT]

print("len(train_val_dataset) =", len(train_val_dataset))
print("len(test_dataset) =", len(test_dataset))
print("len(train_val_labelset) =", len(train_val_labelset))
print("len(test_labelset) =", len(test_labelset))
#'''

'\nTRAIN_TEST_SPLIT = int(len(labelset_stratified)*0.5)\nprint("TRAIN_TEST_SPLIT =", TRAIN_TEST_SPLIT)\n\ntrain_val_dataset = dataset_stratified[TRAIN_TEST_SPLIT:]\ntest_dataset = dataset_stratified[:TRAIN_TEST_SPLIT]\n\ntrain_val_labelset = labelset_stratified[TRAIN_TEST_SPLIT:]\ntest_labelset = labelset_stratified[:TRAIN_TEST_SPLIT]\n\nprint("len(train_val_dataset) =", len(train_val_dataset))\nprint("len(test_dataset) =", len(test_dataset))\nprint("len(train_val_labelset) =", len(train_val_labelset))\nprint("len(test_labelset) =", len(test_labelset))\n#'

In [ ]:
num_folds = 10
split_indices = int(len(labelset_stratified)/num_folds)
fold = 3

train_val_dataset = dataset_stratified[:fold*split_indices-split_indices]
train_val_dataset += dataset_stratified[fold*split_indices:]
test_dataset = dataset_stratified[fold*split_indices-split_indices:fold*split_indices]

train_val_labelset = labelset_stratified[:fold*split_indices-split_indices]
train_val_labelset += labelset_stratified[fold*split_indices:]
test_labelset = labelset_stratified[fold*split_indices-split_indices:fold*split_indices]

print("len(train_val_dataset) =", len(train_val_dataset))
print("len(test_dataset) =", len(test_dataset))
print("len(train_val_labelset) =", len(train_val_labelset))
print("len(test_labelset) =", len(test_labelset))

len(train_val_dataset) = 6300
len(test_dataset) = 700
len(train_val_labelset) = 6300
len(test_labelset) = 700


In [ ]:
np_train_val_dataset = np.array(train_val_dataset)
np_test_dataset = np.array(test_dataset)
np_train_val_labelset = np.array(train_val_labelset)
np_test_labelset = np.array(test_labelset)

print("np_train_val_dataset.shape =", np_train_val_dataset.shape)
print("np_test_dataset.shape =", np_test_dataset.shape)
print("np_train_val_labelset.shape =", np_train_val_labelset.shape)
print("np_test_labelset.shape =", np_test_labelset.shape)

np_train_val_dataset.shape = (6300, 21889)
np_test_dataset.shape = (700, 21889)
np_train_val_labelset.shape = (6300,)
np_test_labelset.shape = (700,)


In [ ]:
from sklearn.utils import shuffle

shuffled_train_val_dataset, shuffled_train_val_labelset = shuffle(np_train_val_dataset, np_train_val_labelset) # , random_state=0

In [ ]:
print(shuffled_train_val_labelset[:10])

[1 0 1 1 0 1 1 0 1 1]


In [ ]:
final_train_dataset = np.expand_dims(shuffled_train_val_dataset, axis = 2)
final_test_dataset = np.expand_dims(np_test_dataset, axis = 2)

y_train2 = np.expand_dims(shuffled_train_val_labelset, axis = 1)
y_test2 = np.expand_dims(np_test_labelset, axis = 1)

print('final_train_dataset.shape =', final_train_dataset.shape)
print('final_test_dataset.shape =', final_test_dataset.shape)
print('y_train2.shape =', y_train2.shape)
print('y_test2.shape =', y_test2.shape)

final_train_dataset.shape = (6300, 21889, 1)
final_test_dataset.shape = (700, 21889, 1)
y_train2.shape = (6300, 1)
y_test2.shape = (700, 1)


In [ ]:
num_classes = 2
final_train_label = tf.keras.utils.to_categorical(y_train2, num_classes)
final_test_label = tf.keras.utils.to_categorical(y_test2, num_classes)

print('final_train_label.shape =',final_train_label.shape)
print('final_test_label.shape =',final_test_label.shape)

final_train_label.shape = (6300, 2)
final_test_label.shape = (700, 2)


In [ ]:
final_train_label[:5]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
#'''
epoch_initial = True
count_no_improvement = 0
#'''

In [ ]:
impl_type = "GRU" # LSTM GRU BiLSTM BiGRU Dense
dataset_name = "BnSent_Bin_Nafis"

NUM_NEURONS = 48 # 32+16
#NUM_DENSE_NEURONS = 0 # x_train.shape[1]*x_train.shape[2]
NUM_LAYERS = 5
BATCH_SIZE = 64
NUM_EPOCHS = 3000
#epochs_completed = 0
LEARNING_RATE = 0.0001
EPSILON = 1e-4
DROPOUT = 0.5
#REC_DROP = 0.5  

'''
ker_init = 'glorot_uniform' # 'glorot_uniform' 'zeros' 'ones'
rec_init = 'orthogonal' # 'orthogonal' 'zeros' 'ones'
b_init = 'zeros' # 'zeros' 'ones'
#'''

OPTIMIZER = "Adam" # "RMSProp" "SGD" "Adam" "Adamax" "Adadelta" "Adagrad" "SGD"
LOSS = 'categorical_crossentropy' # binary_crossentropy categorical_crossentropy
ACTIVATION_FUNCTION = 'default' # 'relu' 'LeakyReLU' 'PReLU' 'ELU' 'default'
DENSE_ACTIVATION_FUNCTION = 'default' # 'relu' 'LeakyReLU' 'PReLU' 'ELU' 'default'
FINAL_ACTIVATION_FUNCTION = 'softmax'
validation_split = 0.1
#kernel_size=(1,1)
early_stop_after_epochs = 5
chk_name_ext = "_withGRU_L1_last_run3_2" # "_2" _3 "_rerun" _withDenseL1_2 _withGRU_L1 _withLSTM_L1 noFlatten _withGRU_L1_last_2 _withGRU_L1_last_split0

# Checkpointer with Dense:
'''
checkpointer_name  = "weights."+dataset_name+"."+impl_type+"."+ACTIVATION_FUNCTION+".dAct."+DENSE_ACTIVATION_FUNCTION+".nLayers"+\
                    str(NUM_LAYERS)+".nNeurons"+str(NUM_NEURONS)+".dNeurons"+str(NUM_DENSE_NEURONS)+".opt"+OPTIMIZER+\
                    ".batch"+str(BATCH_SIZE)+".dropout"+str(DROPOUT)+chk_name_ext+".hdf5"
#'''

# Checkpointer without Dense:
#''' # +".recDrop."+str(REC_DROP)
checkpointer_name  = "weights."+dataset_name+"."+impl_type+".nFolds"+str(num_folds)+".Fold"+str(fold)+\
                    ".act."+ACTIVATION_FUNCTION+".nLayers"+str(NUM_LAYERS)+".nNeurons"+str(NUM_NEURONS)+".opt"+OPTIMIZER+\
                    ".batch"+str(BATCH_SIZE)+".dropout"+str(DROPOUT)+\
                    ".loss."+LOSS+".lr."+str(LEARNING_RATE)+chk_name_ext+".hdf5"
#'''

log_name = "log_"+checkpointer_name[8:-5]+".log"

print('checkpointer_name =', checkpointer_name)
print('log_name =', log_name)

checkpointer_name = weights.BnSent_Bin_Nafis.GRU.nFolds10.Fold3.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run3_2.hdf5
log_name = log_BnSent_Bin_Nafis.GRU.nFolds10.Fold3.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run3_2.log


In [ ]:
def activation_function():
    if ACTIVATION_FUNCTION == 'relu':
        tf.keras.layers.ReLU(max_value=None, negative_slope=0, threshold=0)
    elif ACTIVATION_FUNCTION == 'LeakyReLU':
        tf.keras.layers.LeakyReLU(alpha=0.3)
    elif ACTIVATION_FUNCTION == 'PReLU':
        tf.keras.layers.PReLU(tf.initializers.constant(0.3)) # "zeros"
    elif ACTIVATION_FUNCTION == 'ELU':
        tf.keras.layers.ELU(alpha=1.0)
    elif ACTIVATION_FUNCTION == 'default':
        return 'tanh'

In [ ]:
def dense_activation():
    if DENSE_ACTIVATION_FUNCTION == 'relu':
        tf.keras.layers.ReLU(max_value=None, negative_slope=0, threshold=0)
    elif DENSE_ACTIVATION_FUNCTION == 'LeakyReLU':
        tf.keras.layers.LeakyReLU(alpha=0.3)
    elif DENSE_ACTIVATION_FUNCTION == 'PReLU':
        tf.keras.layers.PReLU(tf.initializers.constant(0.3)) # "zeros"
    elif DENSE_ACTIVATION_FUNCTION == 'ELU':
        tf.keras.layers.ELU(alpha=1.0)
    elif DENSE_ACTIVATION_FUNCTION == 'default':
        return None

In [ ]:
'''
def gru(x):
    return GRU(
        NUM_NEURONS, 
        return_sequences = True, 
        kernel_initializer = ker_init, # ker_init tf.keras.initializers.HeNormal()
        recurrent_initializer = ker_init, # rec_init tf.keras.initializers.HeNormal()
        bias_initializer = b_init,
        dropout = DROPOUT, 
        #recurrent_dropout = REC_DROP,
        activation = activation_function()
        )(x)
#'''

'\ndef gru(x):\n    return GRU(\n        NUM_NEURONS, \n        return_sequences = True, \n        kernel_initializer = ker_init, # ker_init tf.keras.initializers.HeNormal()\n        recurrent_initializer = ker_init, # rec_init tf.keras.initializers.HeNormal()\n        bias_initializer = b_init,\n        dropout = DROPOUT, \n        #recurrent_dropout = REC_DROP,\n        activation = activation_function()\n        )(x)\n#'

In [ ]:
#DROPOUT = 0.8

input_shape = final_train_dataset.shape[1:]

# Input tensor shape
inputs = Input(input_shape)
x = inputs

'''
x = Dense(NUM_DENSE_NEURONS, activation=dense_activation())(inputs)
#x = Dropout(DROPOUT)(x)

for _ in range(NUM_LAYERS):
    x = gru(x)
#'''

#'''
"""
if impl_type == "Dense":
    x = GRU(NUM_NEURONS, return_sequences=True, activation=activation_function())(x)
    #x = LSTM(NUM_NEURONS, return_sequences=True, activation=activation_function())(x)
    x = Dropout(DROPOUT)(x)
    x = Flatten()(x)
#"""
for _ in range(NUM_LAYERS):
    if impl_type == "GRU":
        x = GRU(NUM_NEURONS, return_sequences=True, dropout=DROPOUT, activation=activation_function())(x)
    elif impl_type == "BiGRU":
        x = Bidirectional(GRU(NUM_NEURONS, return_sequences=True, dropout=DROPOUT, activation=activation_function()))(x)
    elif impl_type == "LSTM":
        x = LSTM(NUM_NEURONS, return_sequences=True, dropout=DROPOUT, activation=activation_function())(x)
    elif impl_type == "BiLSTM":
        x = Bidirectional(LSTM(NUM_NEURONS, return_sequences=True, dropout=DROPOUT, activation=activation_function()))(x)
    elif impl_type == "Dense":
        #x = Flatten()(x)
        x = Dense(NUM_NEURONS, activation=dense_activation())(x)
        x = Dropout(DROPOUT)(x)
#'''
#"""
if impl_type == "Dense":
    x = GRU(NUM_NEURONS, return_sequences=True, activation=activation_function())(x)
    #x = LSTM(NUM_NEURONS, return_sequences=True, activation=activation_function())(x)
    x = Dropout(DROPOUT)(x)
    x = Flatten()(x)
#"""
if impl_type != "Dense":
    x = Flatten()(x)

'''
x = Dense(1024, activation=dense_activation())(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation=dense_activation())(x)
x = Dropout(0.5)(x)
#'''
outputs = Dense(num_classes, activation=FINAL_ACTIVATION_FUNCTION)(x)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21889, 1)]        0         
_________________________________________________________________
gru (GRU)                    (None, 21889, 48)         7344      
_________________________________________________________________
gru_1 (GRU)                  (None, 21889, 48)         14112     
_________________________________________________________________
gru_2 (GRU)                  (None, 21889, 48)         14112     
_________________________________________________________________
gru_3 (GRU)                  (None, 21889, 48)         14112     
_________________________________________________________________
gru_4 (GRU)                  (None, 21889, 48)         14112     
_________________________________________________________________
flatten (Flatten)            (None, 1050672)           0     

In [ ]:
# "RMSProp" "SGD" "Adam" "Adamax" "Adadelta" "Adagrad" "SGD"
#optimizer = tf.keras.optimizers.RMSprop(lr = LEARNING_RATE, epsilon=EPSILON)

if OPTIMIZER == "RMSProp":
    optimizer = tf.keras.optimizers.RMSprop(lr = LEARNING_RATE, epsilon=EPSILON)
elif OPTIMIZER == "Adam":
    optimizer = tf.keras.optimizers.Adam(lr = LEARNING_RATE, epsilon=EPSILON, beta_1=0.9, beta_2=0.999)
elif OPTIMIZER == "Adamax":
    optimizer = tf.keras.optimizers.Adamax(lr = LEARNING_RATE, epsilon=EPSILON, beta_1=0.9, beta_2=0.999)
elif OPTIMIZER == "Adadelta":
    optimizer = tf.keras.optimizers.Adadelta(lr = LEARNING_RATE, epsilon=EPSILON, rho=0.95)
elif OPTIMIZER == "Adagrad":
    optimizer = tf.keras.optimizers.Adagrad(lr = LEARNING_RATE, epsilon=EPSILON, initial_accumulator_value=0.1)
elif OPTIMIZER == "SGD":
    optimizer = tf.keras.optimizers.SGD(lr = LEARNING_RATE, momentum=0.85)

'''
list_metrics = ['accuracy', #tf.keras.metrics.Accuracy(), 
                tf.keras.metrics.Precision(), 
                tf.keras.metrics.Recall(),
                tf.keras.metrics.TruePositives(),
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.FalsePositives(),
                tf.keras.metrics.FalseNegatives()]
#'''

model.compile(
    #optimizer=OPTIMIZER,
    optimizer=optimizer,
    loss=LOSS,
    metrics='accuracy' #['accuracy', 'Precision', 'Recall'], list_metrics
)

print("OPTIMIZER =", OPTIMIZER)

OPTIMIZER = Adam


In [ ]:
# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath = work_dir+checkpointer_name, 
                               #monitor='val_accuracy',
                               monitor='val_loss',
                               save_weights_only=False,  
                               mode='auto', 
                               verbose = 0, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                               save_best_only =False
                               )
checkpointer_best = ModelCheckpoint(filepath = work_dir+"best_"+checkpointer_name, 
                                    monitor='val_loss', 
                                    save_weights_only=False,
                                    mode='auto',  
                                    verbose = 1, 
                                    save_best_only = True
                                    )
early_stopping = EarlyStopping(monitor='loss', patience=early_stop_after_epochs)

In [ ]:
'''
if 'count_no_improvement' not in globals():
    count_no_improvement = 0
    print("count_no_improvement =", count_no_improvement)
#'''
'''
count_no_improvement = 0
epoch_initial = False
#'''

min_delta = 0.0001
print("count_no_improvement =", count_no_improvement)

def checkBestPerformance(epoch, logs):
    save_filepath = work_dir+"best_"+checkpointer_name
    
    global epoch_initial
    if epoch_initial == True:
        epoch_initial = False
        model.save(filepath = save_filepath)
        print(". Model saved!")

    elif epoch_initial == False:
        global count_no_improvement

        log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['val_loss', 'val_accuracy'], engine='python')
        min_val_loss = float(str(min(log_data.val_loss.values))[:6])
        max_val_acc = float(str(max(log_data.val_accuracy.values))[:6])

        current_val_acc = float(str(logs['val_accuracy'])[:6])
        current_val_loss = float(str(logs['val_loss'])[:6])

        if (current_val_loss < min_val_loss) and (abs(current_val_loss-min_val_loss) >= min_delta):
            count_no_improvement = 0
            model.save(filepath = save_filepath)
            print("\nval_loss decreased from",min_val_loss," to",current_val_loss,"( val_accuracy =",current_val_acc,").")

        elif (current_val_loss==min_val_loss) and (current_val_acc>max_val_acc):
            count_no_improvement = 0
            model.save(filepath = save_filepath)
            print("\nval_accuracy increased to", current_val_acc, ".")

        else:
            count_no_improvement += 1
            print(". count_no_improvement =", count_no_improvement)

        if count_no_improvement >= early_stop_after_epochs:
            global list_callbacks
            del list_callbacks, count_no_improvement
            #print("count_no_improvement =", count_no_improvement, "... list_callbacks =", list_callbacks)

count_no_improvement = 0


In [ ]:
epochs_completed = 0
list_callbacks = []
csv_logger = CSVLogger(work_dir+log_name, separator=',', append=True)

#if 'list_callbacks' in globals():
#    del list_callbacks

try:
    log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['epoch'], engine='python')
    epochs_completed = log_data.shape[0]

    #if epochs_completed > 0:
    model = load_model(work_dir+checkpointer_name)
    list_callbacks = [checkpointer, LambdaCallback(on_epoch_end=checkBestPerformance), csv_logger]
    print("epochs_completed =", epochs_completed)
    
except Exception as error:
    if epochs_completed == 0:
        # list_callbacks = [checkpointer, checkpointer_best, csv_logger, early_stopping] 
        list_callbacks = [checkpointer, LambdaCallback(on_epoch_end=checkBestPerformance), csv_logger]
        print("epochs_completed =", epochs_completed)
    elif epochs_completed > 0:
        print(error)

print('checkpointer_name =', checkpointer_name)

epochs_completed = 0
checkpointer_name = weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold3.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run3.hdf5


In [ ]:
print('checkpointer_name =', checkpointer_name)
print("Previously completed epochs =", epochs_completed)
print("count_no_improvement =", count_no_improvement, "\n")

#'''
try:
    start_time = time.time()
    history = model.fit(final_train_dataset, final_train_label, 
                        shuffle=True, 
                        batch_size = BATCH_SIZE, 
                        epochs = NUM_EPOCHS - epochs_completed, 
                        #steps_per_epoch = 2,
                        #validation_split = validation_split, 
                        validation_data = (final_test_dataset, final_test_label),
                        callbacks=list_callbacks
                        )
    elapsed_time = time.time() - start_time 
    print("\nTime elapsed: ", elapsed_time)

except Exception as error:
    print("\nError:", error)
#'''

checkpointer_name = weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold3.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run3.hdf5
Previously completed epochs = 0
count_no_improvement = 0 

Epoch 1/3000
99/99 [==============================] - 82s 825ms/step - loss: 0.6913 - accuracy: 0.5335 - val_loss: 0.6832 - val_accuracy: 0.5329
Epoch 2/3000
99/99 [==============================] - ETA: 0s - loss: 0.6727 - accuracy: 0.6668
val_loss decreased from 0.6832  to 0.6281 ( val_accuracy = 0.7657 ).
99/99 [==============================] - 82s 832ms/step - loss: 0.6727 - accuracy: 0.6668 - val_loss: 0.6281 - val_accuracy: 0.7657
Epoch 3/3000
99/99 [==============================] - ETA: 0s - loss: 0.5783 - accuracy: 0.7889
val_loss decreased from 0.6281  to 0.4754 ( val_accuracy = 0.8299 ).
99/99 [==============================] - 82s 825ms/step - loss: 0.5783 - accuracy: 0.7889 - val_loss: 0.4755 - val_accuracy: 0.8300
Epoch 4/300

In [ ]:
csv_logger = CSVLogger(work_dir+log_name, separator=',', append=True)
log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['epoch'], engine='python')
epochs_completed = log_data.shape[0]

In [ ]:
model_loaded = load_model(work_dir+"best_"+checkpointer_name)
print("Loaded "+work_dir+"best_"+checkpointer_name+".")

In [ ]:
'''
Test Acc: 0.7143, Test Loss: 0.5672: ep12, best_weights.BanglaSentiment_Bin_NafisAlvi.GRU.act.default.nLayers3.nNeurons16.optAdam.batch64.dropout0.5.loss.categorical_crossentropy_withDenseL1_2.hdf5
Test Acc: 0.6986, Test Loss: 0.5704: ep9,  best_weights.BanglaSentiment_Bin_NafisAlvi.LSTM.act.default.nLayers3.nNeurons16.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.hdf5
Test Acc: 0.6800, Test Loss: 0.6241: ep9,  best_weights.BanglaSentiment_Bin_NafisAlvi.BiGRU.act.default.nLayers3.nNeurons16.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.hdf5
-
Test Acc: 0.6643, Test Loss: 0.6049: ep18, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers3.nNeurons16.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.hdf5
Test Acc: 0.6771, Test Loss: 0.6042: ep12, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers3.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.hdf5
Test Acc: 0.6900, Test Loss: 0.5965: ep16, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optRMSProp.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6829, Test Loss: 0.5981: ep17, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optRMSProp.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_2.hdf5
Test Acc: 0.6843, Test Loss: 0.6070: ep147, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optSGD.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_2.hdf5
Test Acc: 0.6771, Test Loss: 0.6079: ep101, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optSGD.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_3.hdf5
Test Acc: 0.6800, Test Loss: 0.6082: ep138, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons64.optSGD.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6743, Test Loss: 0.6044: ep220, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers3.nNeurons64.optSGD.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6714, Test Loss: 0.6100: ep18, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons64.optRMSProp.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6771, Test Loss: 0.6083: ep16, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons128.optRMSProp.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6757, Test Loss: 0.6049: ep139, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons128.optSGD.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6757, Test Loss: 0.6237: ep14, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optRMSProp.batch8.dropout0.5.loss.categorical_crossentropy.lr.0.0001.hdf5
Test Acc: 0.6829, Test Loss: 0.5969: ep9, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001noFlatten.hdf5

-
Test Acc: 0.7371, Test Loss: 0.5529: ep13, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optRMSProp.batch8.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1.hdf5
Test Acc: 0.7314, Test Loss: 0.5575: ep11, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optRMSProp.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1.hdf5
Test Acc: 0.7129, Test Loss: 0.5963: ep11, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optRMSProp.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L3.hdf5
Test Acc: 0.7429, Test Loss: 0.5550: ep9,  best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1.hdf5
Test Acc: 0.7071, Test Loss: 0.5634: ep8,  best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withLSTM_L1.hdf5
Test Acc: 0.7614, Test Loss: 0.5159: ep11, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last.hdf5
Test Acc: 0.7557, Test Loss: 0.4976: ep11, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_2.hdf5
Test Acc: 0.7657, Test Loss: 0.5428: ep7,  best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch8.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_2.hdf5
Test Acc: 0.7386, Test Loss: 0.5313: ep11, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons32.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_3.hdf5
Test Acc: 0.7514, Test Loss: 0.5138: ep10, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last.hdf5
-
Test Acc: 0.8229, Test Loss: 0.4319: ep12, best_weights.BanglaSentiment_Bin_NafisAlvi.Dense.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_split0.5.hdf5

Test Acc: 0.8029, Test Loss: 0.4255: ep12, best_weights.BnSent_Bin_Nafis.Dense.nFolds.2.Fold.1.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1_2.hdf5
Test Acc: 0.6703, Test Loss: 0.6366: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds.2.Fold.2.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1_2.hdf5

Test Acc: 0.7614, Test Loss: 0.5033: ep9, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold1.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.8514, Test Loss: 0.3924: ep11, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold2.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.9071, Test Loss: 0.2626: ep13, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold3.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.8471, Test Loss: 0.327: ep11, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold4.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.8686, Test Loss: 0.3193: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold5.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.8671, Test Loss: 0.3429: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold6.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.8214, Test Loss: 0.4216: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold7.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.6214, Test Loss: 0.676: ep7, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold8.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.7871, Test Loss: 0.5899: ep13, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold9.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Test Acc: 0.5086, Test Loss: 0.6971: ep6, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold10.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run1.hdf5
Avg Acc: 0.78412, Avg Loss: 0.45321.
-
Test Acc: 0.7371, Test Loss: 0.576: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold1.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.8486, Test Loss: 0.3868: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold2.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.8986, Test Loss: 0.2828: ep12, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold3.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.8529, Test Loss: 0.3138: ep12, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold4.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.87, Test Loss: 0.3458: ep11, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold5.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.8457, Test Loss: 0.3582: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold6.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.8143, Test Loss: 0.4031: ep9, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold7.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.6386, Test Loss: 0.6811: ep8, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold8.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.7171, Test Loss: 0.5966: ep8, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold9.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5
Test Acc: 0.5814, Test Loss: 0.6883: ep8, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold10.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run2.hdf5

-
Test Acc: 0.7214, Test Loss: 0.5551: ep9, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold1.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run3.hdf5
Test Acc: 0.8571, Test Loss: 0.3614: ep10, best_weights.BnSent_Bin_Nafis.Dense.nFolds10.Fold2.act.default.nLayers5.nNeurons48.optAdam.batch64.dropout0.5.loss.categorical_crossentropy.lr.0.0001_withGRU_L1_last_run3.hdf5

'''

result2 = model_loaded.evaluate(final_test_dataset, final_test_label)
#print("nLayers: {}, nNeurons: {}, DROPOUT: {}, Test Acc: {}, Test Loss: {}".format(NUM_LAYERS, NUM_NEURONS, DROPOUT, round(result2[1], 4), round(result2[0], 4)))
print("Test Acc: {}, Test Loss: {}: ep{}, {}\n".format(round(result2[1],4), round(result2[0],4), epochs_completed, "best_"+checkpointer_name))


In [ ]:
with open(work_dir+'Records.csv', "a") as fp:
    wr = csv.writer(fp, dialect='excel')
    try:
        wr.writerow([checkpointer_name[8:-5], round(result2[1], 4), round(result2[0], 4), elapsed_time])
    except:
        wr.writerow([checkpointer_name[8:-5], round(result2[1], 4), round(result2[0], 4)])
print("Saved results.")

In [ ]:
'''
#Confution Matrix and Classification Report
Y_pred = model_loaded.predict_generator(np_test_dataset, len(np_test_dataset))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(np_test_labelset, y_pred))
#'''

In [ ]:
'''
print('Classification Report:')
#target_names = ['Mono', 'Di'] # not ['Di', 'Mono']
print(classification_report(np_test_labelset, y_pred)) #, target_names=target_names))
#'''

In [ ]:
log_data = pd.read_csv(work_dir+log_name, sep=',', engine='python') 
print("log_data.shape =", log_data.shape)

In [ ]:
log_data.head()

In [ ]:
'''
# plot the training artifacts
title = "Val loss for "+dataset+" "+impl_type+" ("+str(NUM_LAYERS)+" layers, "+str(NUM_NEURONS)+" neurons)"

plt.plot(log_data['loss'])
plt.plot(log_data['val_loss'])
plt.title(title)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'best')

img_path = work_dir+'Images/vLoss_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)
#'''

In [ ]:
'''
title = "Val acc for "+dataset+" "+impl_type+" ("+str(NUM_LAYERS)+" layers, "+str(NUM_NEURONS)+" neurons)"

plt.plot(log_data['accuracy'])
plt.plot(log_data['val_accuracy'])
plt.title(title)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'], loc = 'best')

img_path = work_dir+'Images/vAcc_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)
#'''